# 【確認】3-1-eの記述を確認

In [ ]:
import numpy as np

from section3_1_heatingday import \
    get_heating_flag_d

import section3_1_e as algo

from section11_2 import \
    get_I_s_d_t, \
    get_Theta_ex

# 第九章　自然エネルギー利用設備

## 第三節　空気集熱式太陽熱利用設備

1．適用範囲～4．記号及び単位は省略

### 5．暖房負荷削減量

集熱後の空気を床下空間へ供給する場合： 

$L'_{H,R,d,t,i} \leq 0$ または 当該時刻において集熱した熱を暖房に利用しない場合 ($r_{sa,d,t} = 0$) の場合： 


$$\large
\begin{equation*}
    \Delta L'_{H,ass,d,t,i} = 0
\end{equation*}
$$

$0 < L'_{H,R,d,t,i}$ かつ 当該時刻において集熱した熱を暖房に利用する場合 ($0 < r_{sa,d,t}$) の場合： 


$$\large
\begin{equation*}
    \Delta L'_{H,ass,d,t,i} = \rho_{air} \times c_{p_{air}} \times V_{sa,d,t,i} \times (\theta_{uf,d,t} - \theta_{in,d,t}) \times 10^{-3}\\ 
    - U_s \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{uf,d,t}) \times 3.6 \times 10^{-3}\\
    + U_{s,vert} \times A_{s,ufvnt,i} \times (\theta_{in,d,t} - \theta_{ex,d,t}) \times H_{floor} \times 3.6 \times 10^{-3}
\end{equation*}
$$

集熱後の空気を居室へ直接供給する場合： 

$L'_{H,R,d,t,i} \leq 0$ または 当該時刻において集熱した熱を暖房に利用しない場合 ($r_{sa,d,t} = 0$) の場合： 


$$\large
\begin{equation*}
    \Delta L'_{H,ass,d,t,i} = 0
\end{equation*}
$$

$0 < L'_{H,R,d,t,i}$ かつ 当該時刻において集熱した熱を暖房に利用する場合 ($0 < r_{sa,d,t}$) の場合： 


$$\large
\begin{equation*}
    \Delta L'_{H,ass,d,t,i} = \rho_{air} \times c_{p_{air}} \times V_{sa,d,t,i} \times (\theta_{sa,d,t} - \theta_{in,d,t}) \times 10^{-3}
\end{equation*}
$$

$A_{s,ufvnt,i}$：暖冷房区画$i$の空気を供給する床下空間に接する床の面積(m<sup>2</sup>)  
$c_{p_{air}}$： 空気の比熱(kJ/(kg・K))   
$H_{floor}$：床の温度差係数(-)  
$L'_{H,R,d,t,i}$：日付$d$の時刻$t$における標準住戸の暖冷房区画$i$の負荷補正前の暖房負荷(MJ/h)  
$r_{sa,d,t}$：日付$d$の時刻$t$における 1 時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する空気の風量の割合(-)  
$U_s$：床の熱貫流率(W/(m<sup>2</sup>・K))  
$U_{s,vert}$：暖冷房負荷計算時に想定した床の熱貫流率(W/(m<sup>2</sup>・K))  
$V_{sa,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1 時間当たりの床下空間または居室へ供給する空気の風量(m<sup>3</sup>/h)  
$\Delta L'_{H,ass,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の空気集熱式太陽熱利用設備による暖房負荷削減量(MJ/h)  
$\rho_{air}$：空気の密度(kg/m<sup>3</sup>)  
$\theta_{in,d,t}$：日付$d$の時刻$t$における室内温度(℃)  
$\theta_{sa,d,t}$：日付$d$の時刻$t$において床下空間または居室へ供給する空気の温度(℃)  
$\theta_{uf,d,t}$：日付$d$の時刻$t$における床下温度(℃)  

In [ ]:
# 空気集熱式太陽熱利用設備による暖房負荷削減量 (MJ/h)
def calc_delta_L_dash_H_ass_d_t_i(i, supply_target, L_dash_H_R_d_t_i, region, sol_region, A_HCZ_i, A_A, A_MR, A_OR, Q,
                                  hotwater_use, Theta_ex_d_t, P_alpha, P_beta,
                                  A_col, V_fan_P0, m_fan_test, d0, d1, ufv_insulation, r_A_ufvnt):
    """
    :param i: 暖冷房区画の番号
    :param supply_target: 集熱後の空気を供給する先
    :param L_dash_H_R_d_t_i:
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :param A_HCZ_i: 暖冷房区画iの床面積 (m2)
    :param A_A:
    :param A_MR:
    :param A_OR:
    :param Q:
    :param Theta_sa_d_t: 床下空間または居室へ供給する空気の温度 (℃)
    :param Theta_ex_d_t: 外気温度 (℃)
    :param P_alpha:
    :param P_beta:
    :param A_col:
    :param V_fan_P0: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとしたときの空気搬送ファンの風量 (m3/h)
    :param m_fan_test:
    :param d0:
    :param d1:
    :param ufv_insulation:
    :param r_A_ufvnt:
    :return:
    """
    # ----- 10 集熱部 -----

    # 空気搬送ファン停止時および稼働時の集熱部の出口における空気温度 (℃)
    Theta_col_nonopg_d_t, Theta_col_opg_d_t = calc_Theta_col(A_col=A_col, P_alpha=P_alpha, P_beta=P_beta,
                                                             V_fan_P0=V_fan_P0, d0=d0, d1=d1, m_fan_test=m_fan_test,
                                                             region=region, sol_region=sol_region,
                                                             Theta_ex_d_t=Theta_ex_d_t)

    # ----- 9. 空気搬送部 -----

    # 1時間当たりの空気搬送ファンの稼働時間 (h/h) (12)
    t_fan_d_t = get_t_fan_d_t(Theta_col_nonopg_d_t, Theta_col_opg_d_t)

    # 1時間当たりの空気搬送ファンの風量 (m3/h) (13)
    V_fan_d_t = get_V_fan_d_t(t_fan_d_t, V_fan_P0)

    # 暖房日
    heating_flag_d = get_heating_flag_d(L_dash_H_R_d_t_i)

    # 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (17)
    r_sa_d_t = get_r_sa_d_t(t_fan_d_t, heating_flag_d)

    # 床下空間または居室へ供給する1時間当たりの空気の風量の合計 (m3/h) (16)
    V_sa_d_t_A = get_V_sa_d_t_A(V_fan_d_t, r_sa_d_t)

    # 1時間当たりの床下空間または居室へ供給する空気の風量 (15)
    V_sa_d_t_i = get_V_sa_d_t_i(i, A_HCZ_i, A_MR, A_OR, V_sa_d_t_A)

    # ----- 5. 暖房負荷削減量 -----

    # 当該住戸の暖冷房区画iの外気を導入する床下空間に接する床の面積
    if supply_target == '床下':
        A_s_ufvnt_i = algo.get_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR)
    else:
        A_s_ufvnt_i = None

    # 暖冷房負荷計算時に想定した床の熱貫流率 (W/m2K)
    U_s_vert = algo.get_U_s_vert(region, Q)

    # 1時間当たりの集熱部における集熱量 (MJ/h) (19)
    Q_col_d_t = get_Q_col_d_t(V_fan_d_t, Theta_col_opg_d_t, Theta_ex_d_t)

    # 1時間当たりの循環ポンプの稼働時間 (h/h) (6)
    t_cp_d_t = get_t_cp_d_t(hotwater_use, t_fan_d_t, heating_flag_d)

    # 1時間当たりの集熱部における集熱量のうちの給湯利用分 (MJ/h) (11)
    Q_col_W_d_t = get_Q_col_w_d_t(Q_col_d_t, t_cp_d_t)

    # 床下空間または居室へ供給する空気の温度 (℃) (18)
    Theta_sa_d_t = get_Theta_sa_d_t(V_fan_d_t, Theta_col_opg_d_t, Q_col_W_d_t)

    # 床下温度及び地盤またはそれを覆う基礎の表面温度 (℃)
    if supply_target == '床下':
        Theta_uf_d_t, Theta_g_surf_d_t = algo.calc_Theta(
            region=region,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            Q=Q,
            H_OR_C='H',
            # 床下の設定
            r_A_ufvnt=r_A_ufvnt,
            underfloor_insulation=ufv_insulation,
            V_sa_d_t_A=V_sa_d_t_A,
            # 温度・負荷
            Theta_sa_d_t=Theta_sa_d_t,
            Theta_ex_d_t=Theta_ex_d_t,
            L_dash_H_R_d_t=L_dash_H_R_d_t_i,
        )
    else:
        Theta_uf_d_t, Theta_g_surf_d_t = None, None

    # 空気集熱式太陽熱利用設備による暖房負荷削減量 (1)
    delta_L_dash_H_ass_d_t_i = get_delta_L_dash_H_ass_d_t_i(
        # 床下の設定
        supply_target=supply_target,
        A_s_ufvnt_i=A_s_ufvnt_i,
        U_s_vert=U_s_vert,
        V_sa_d_t_i=V_sa_d_t_i,
        r_sa_d_t=r_sa_d_t,
        # 温度・負荷
        Theta_uf_d_t=Theta_uf_d_t,
        Theta_ex_d_t=Theta_ex_d_t,
        Theta_sa_d_t=Theta_sa_d_t,
        L_dash_H_R_d_t_i=L_dash_H_R_d_t_i[i - 1],
    )

    return delta_L_dash_H_ass_d_t_i


# 空気集熱式太陽熱利用設備による暖房負荷削減量 (1)
def get_delta_L_dash_H_ass_d_t_i(A_s_ufvnt_i, L_dash_H_R_d_t_i, Theta_ex_d_t, Theta_sa_d_t, Theta_uf_d_t, U_s_vert,
                                 V_sa_d_t_i, r_sa_d_t, supply_target):
    """
    :param A_s_ufvnt_i: 暖冷房区画iの空気を供給する床下空間に接する床の面積 (m2)
    :param L_dash_H_R_d_t_i: 標準住戸の暖冷房区画iの負荷補正前の暖房負荷 (MJ/h)
    :param Theta_ex_d_t: 外気温度 (℃)
    :param Theta_sa_d_t: 床下空間または居室へ供給する空気の温度 (℃)
    :param Theta_uf_d_t: 当該住戸の床下温度 (℃)
    :param U_s_vert: 暖冷房負荷計算時に想定した床の熱貫流率 (W/m2K)
    :param V_sa_d_t_i: 1時間当たりの床下空間または居室へ供給する空気の風量 (m3/h)
    :param r_sa_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    :param supply_target: 集熱後の空気を供給する先
    :return: 空気集熱式太陽熱利用設備による暖房負荷削減量 (MJ/h)
    """

    # 床の温度差係数( -)
    H_floor = get_H_floor()

    # 暖房設定温度 (℃)
    Theta_prst_H = get_Theta_prst_H()

    # 室内温度 (℃)
    Theta_in_d_t = get_Theta_in_d_t(Theta_prst_H)

    # 空気の密度[kg/m3]
    ro_air = get_ro_air()

    # 空気の比熱[kJ/(kgK)]
    c_p_air = get_c_p_air()

    # 床下の熱貫流率[W/m2K]
    U_s = get_U_s()

    if supply_target == '床下':
        # 集熱後の空気を床下空間へ供給する場合

        # (1-2a)
        delta_L_dash_H_ass_d_t_i = \
            ro_air * c_p_air * V_sa_d_t_i * (Theta_uf_d_t - Theta_in_d_t) * 10 ** (-3) \
            - U_s * A_s_ufvnt_i * (Theta_in_d_t - Theta_uf_d_t) * 3.6 * 10 ** (-3) \
            + U_s_vert * A_s_ufvnt_i * (Theta_in_d_t - Theta_ex_d_t) * H_floor * 3.6 * 10 ** (-3)

        # (1-1a)
        delta_L_dash_H_ass_d_t_i[np.logical_or(L_dash_H_R_d_t_i <= 0, r_sa_d_t == 0)] = 0.0

    elif supply_target == '居室':
        # 集熱後の空気を居室へ直接供給する場合

        # (1-2b)
        delta_L_dash_H_ass_d_t_i = \
            ro_air * c_p_air * V_sa_d_t_i * (Theta_sa_d_t - Theta_in_d_t) * 10 ** (-3)

        # (1-2a)
        delta_L_dash_H_ass_d_t_i[np.logical_or(L_dash_H_R_d_t_i <= 0, r_sa_d_t == 0)] = 0.0

    else:
        raise ValueError(supply_target)

    return delta_L_dash_H_ass_d_t_i


def get_delta_L_dash_H_uf_d_t_i():
    return np.zeros(24 * 365)


def get_delta_L_dash_CS_uf_d_t_i():
    return np.zeros(24 * 365)

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup> とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。日付$d$の時刻$t$における室内温度$\theta_{in,d,t}$は、暖房設定温度$\theta_{prst,H}$に等しいとして20℃とする。床の熱貫流率$U_s$は、2.223W/(m<sup>2</sup>・K)とする。床の温度差係数$H_{floor}$は、0.7 とする。

In [ ]:
# 床の温度差係数 (-)
def get_H_floor():
    return 0.7


# 暖房設定温度 (℃)
def get_Theta_prst_H():
    return 20.0


# 室内温度 (℃)
def get_Theta_in_d_t(Theta_prst_H):
    """
    :param Theta_prst_H: 暖房設定温度 (℃)
    :return: 1時間当たりの室内温度 (℃)
    """
    return np.repeat(Theta_prst_H, 24 * 365)


# 空気の密度 (kg/m3)
def get_ro_air():
    return 1.20


# 空気の比熱 (kJ/(kgK))
def get_c_p_air():
    return 1.006


# 床下の熱貫流率 (W/m2K)
def get_U_s():
    return 2.223

### 6．補正集熱量

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} = 0$ の場合：

$$\large
L_{sun,ass,d,t} = 0
$$

$L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} > 0$ の場合：

$$\large
L_{sun,ass,d,t} = L_{sun,ass,d} \times \frac{L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t}}{L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d}}
$$

$L_{sun,ass,d}$ : 日付$d$における1日当たりの空気集熱式太陽熱利用設備による補正集熱量(MJ/d)  
$L_{sun,ass,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気集熱式太陽熱利用設備による補正集熱量(MJ/h)  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d)  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d)  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d)  
$L'_{k,d,t}$ : 日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h)  
$L'_{s,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h)  
$L'_{w,d,t}$ : 日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h)  
$L'_{b1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h)  
$L'_{b2,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h)  
$L'_{ba1,d,t}$ : 日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h)

In [ ]:
# 1時間当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d) (2)
def get_L_sun_ass_d_t(L_tnk_d, L_dash_k_d_t, L_dash_s_d_t, L_dash_w_d_t, L_dash_b1_d_t, L_dash_b2_d_t, L_dash_ba1_d_t):
    """
    :param L_tnk_d: 1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量 (MJ/d)
    :param L_dash_k_d_t: 1時間当たりの台所水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_s_d_t: 1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_w_d_t: 1時間当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_b2_d_t: 1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷 (MJ/h)
    :param L_dash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/h)
    :return: 1時間当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/h)
    """
    # 1日あたりの節湯補正給湯熱負荷 (MJ/d)
    L_dash_k_d = np.sum(L_dash_k_d_t.reshape(365, 24), axis=1)
    L_dash_s_d = np.sum(L_dash_s_d_t.reshape(365, 24), axis=1)
    L_dash_w_d = np.sum(L_dash_w_d_t.reshape(365, 24), axis=1)
    L_dash_b1_d = np.sum(L_dash_b1_d_t.reshape(365, 24), axis=1)
    L_dash_b2_d = np.sum(L_dash_b2_d_t.reshape(365, 24), axis=1)
    L_dash_ba1_d = np.sum(L_dash_ba1_d_t.reshape(365, 24), axis=1)

    # 1日当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d) (3)
    L_sun_ass_d = get_L_sun_ass_d(L_tnk_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d, L_dash_b2_d, L_dash_ba1_d)
    L_sun_ass_d = np.repeat(L_sun_ass_d, 24)

    # 1日当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d)
    L_sun_ass_d_t = np.zeros(24 * 365)

    # 24時間化
    L_dash_k_d = np.repeat(L_dash_k_d, 24)
    L_dash_s_d = np.repeat(L_dash_s_d, 24)
    L_dash_w_d = np.repeat(L_dash_w_d, 24)
    L_dash_b1_d = np.repeat(L_dash_b1_d, 24)
    L_dash_b2_d = np.repeat(L_dash_b2_d, 24)
    L_dash_ba1_d = np.repeat(L_dash_ba1_d, 24)

    # 1) 1日あたりの節湯補正給湯熱負荷 = 0 の場合
    f1 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) == 0
    L_sun_ass_d_t[f1] = 0

    # 2) 1日あたりの節湯補正給湯熱負荷 > 0 の場合
    f2 = (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) > 0
    L_dash_d = L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d
    L_dash_d_t = L_dash_k_d_t + L_dash_s_d_t + L_dash_w_d_t + L_dash_b1_d_t + L_dash_b2_d_t + L_dash_ba1_d_t
    L_sun_ass_d_t[f2] = L_sun_ass_d[f2] * L_dash_d_t[f2] / L_dash_d[f2]

    return L_sun_ass_d_t

$$\large
\begin{equation*}
    L_{sun,ass,d} = min \left( L_{tunk,d} , \: \left( L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} \right) \times f_{sr,uplim} \right)
\end{equation*}
$$

$f_{sr,uplim}$ : 給湯部の分担率上限値(-)   
$L_{sun,ass,d}$ : 日付$d$における1日当たりの空気集熱式太陽熱利用設備による補正集熱量(MJ/d)  
$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/d)  
$L'_{k,d}$ : 日付$d$における1日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{s,d}$ : 日付$d$における1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{w,d}$ : 日付$d$における1日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d)  
$L'_{b1,d}$ : 日付$d$における1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d)  
$L'_{b2,d}$ : 日付$d$における1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d)  
$L'_{ba1,d}$ : 日付$d$における1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d)

In [ ]:
# 1日当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d) (3)
def get_L_sun_ass_d(L_tnk_d, L_dash_k_d, L_dash_s_d, L_dash_w_d, L_dash_b1_d, L_dash_b2_d, L_dash_ba1_d):
    """
    :param L_tnk_d: 1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量 (MJ/d)
    :param L_dash_k_d: 1日当たりの台所水栓における節湯補正給湯熱負荷 (MJ/d)
    :param L_dash_s_d: 1日当たりの浴室シャワー水栓における節湯補正給湯熱負荷 (MJ/d)
    :param L_dash_w_d: 1日当たりの洗面水栓における節湯補正給湯熱負荷 (MJ/d)
    :param L_dash_b1_d: 1日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷 (MJ/d)
    :param L_dash_b2_d: 1日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷 (MJ/d)
    :param L_dash_ba1_d: 1日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷 (MJ/d)
    :return: 1日当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d)
    """
    # 給湯部の分担率上限値
    f_sr_uplim = get_f_sr_uplim()

    # 1日当たりの空気集熱式太陽熱利用設備における補正集熱量 (MJ/d)
    L_sun_ass_d = np.clip(
        L_tnk_d,
        None,
        (L_dash_k_d + L_dash_s_d + L_dash_w_d + L_dash_b1_d + L_dash_b2_d + L_dash_ba1_d) * f_sr_uplim
    )

    return L_sun_ass_d

給湯部の分担率上限値$f_{sr,uplim}$は、0.9とする。 

In [ ]:
# 給湯部の分担率上限値 (-)
def get_f_sr_uplim():
    return 0.9

### 7．補機の消費電力量

当該時刻において集熱した熱を暖房と給湯とのどちらにも利用しない場合( $r_{sa,d,t} = 0$ かつ $t_{cp,d,t} = 0$ )： 

$$\large
E_{E,H,aux,ass,d,t} = 0
$$

$$\large
E_{E,W,aux,ass,d,t} = 0
$$

当該時刻において集熱した熱を暖房のみに利用する場合( $0 < r_{sa,d,t}$ かつ $t_{cp,d,t} = 0$ )： 

$$\large
E_{E,H,aux,ass,d,t} = E_{E,fan,d,t}
$$

$$\large
E_{E,W,aux,ass,d,t} = 0
$$

当該時刻において集熱した熱を給湯のみに利用する場合( $r_{sa,d,t} = 0$ かつ $ 0 < t_{cp,d,t}$ )： 

$$\large
E_{E,H,aux,ass,d,t} = 0
$$

$$\large
E_{E,W,aux,ass,d,t} = E_{E,fan,d,t} + E_{E,cp,d,t}
$$

$E_{E,H,aux,ass,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち暖房設備への付加分(kWh/h)   
$E_{E,W,aux,ass,d,t}$ : 日付$d$の時刻$t$における1時間当たりの補機の消費電力量のうち給湯設備への付加分(kWh/h)   
$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)   
$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)      
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)    
$r_{sa,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する空気の風量の割合(-)   

In [ ]:
# 1時間当たりの補機の消費電力量のうちの給湯設備への付加分 (kWh/h)
def calc_E_E_W_aux_ass_d_t(hotwater_use, heating_flag_d, region, sol_region, P_alpha, P_beta,
                           A_col, V_fan_P0, m_fan_test, d0, d1, fan_sso, fan_type, pump_sso):
    """
    :param hotwater_use: 空気集熱式太陽熱利用設備が給湯部を有する場合はTrue
    :param heating_flag_d: 暖房日
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :param P_alpha: 方位角 (°)
    :param P_beta: 傾斜角 (°)
    :param A_col: 集熱器群の面積 (m2)
    :param V_fan_P0: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとしたときの空気搬送ファンの風量 (m3/h)
    :param d0: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の切片 (-)
    :param d1: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の傾き (W/(m2K))
    :param m_fan_test: 集熱器群を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量 (kg/(s・m2))
    :param fan_sso:
    :param fan_type: ファンの種別
    :param pump_sso: 循環ポンプの自立運転用太陽光発電装置を採用する場合はTrue
    :return: 1時間当たりの補機の消費電力量のうちの給湯設備への付加分 (kWh/h)
    """

    from section11_1 import load_outdoor, get_Theta_ex
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # ----- 10 集熱部 -----

    # 空気搬送ファン停止時および稼働時の集熱部の出口における空気温度 (℃)
    Theta_col_nonopg_d_t, Theta_col_opg_d_t = calc_Theta_col(A_col=A_col, P_alpha=P_alpha, P_beta=P_beta,
                                                             V_fan_P0=V_fan_P0, d0=d0, d1=d1, m_fan_test=m_fan_test,
                                                             region=region, sol_region=sol_region,
                                                             Theta_ex_d_t=Theta_ex_d_t)

    # ----- 9. 空気搬送部 -----

    # 1時間当たりの空気搬送ファンの稼働時間 (h/h) (12)
    t_fan_d_t = get_t_fan_d_t(Theta_col_nonopg_d_t, Theta_col_opg_d_t)

    # 1時間当たりの空気搬送ファンの風量 (m3/h) (13)
    V_fan_d_t = get_V_fan_d_t(t_fan_d_t, V_fan_P0)

    # 1時間当たりの空気搬送ファンの消費電力量 (kWh/h) (14)
    E_E_fan_d_t = get_E_E_fan_d_t(fan_sso, fan_type, V_fan_d_t, t_fan_d_t)

    # 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (17)
    r_sa_d_t = get_r_sa_d_t(t_fan_d_t, heating_flag_d)

    # ----- 8. 給湯部 -----

    # 1時間当たりの循環ポンプの稼働時間 (h/h) (6)
    t_cp_d_t = get_t_cp_d_t(hotwater_use, t_fan_d_t, heating_flag_d)

    # 1時間当たりの循環ポンプの消費電力量 (kWh/h) (7)
    E_E_cp_d_t = get_E_E_cp_d_t(pump_sso, t_cp_d_t)

    # ----- 7. 補機の消費電力量 -----

    # 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h) (4)
    E_E_W_aux_ass_d_t = get_E_E_W_aux_ass_d_t(r_sa_d_t, t_cp_d_t, E_E_fan_d_t, E_E_cp_d_t)

    return E_E_W_aux_ass_d_t


# 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h) (4)
def calc_E_E_H_aux_ass_d_t(hotwater_use, heating_flag_d, region, sol_region, P_alpha, P_beta,
                           A_col, V_fan_P0, m_fan_test, d0, d1, fan_sso, fan_type):
    """
    :param hotwater_use: 空気集熱式太陽熱利用設備が給湯部を有する場合はTrue
    :param heating_flag_d: 暖房日
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :param P_alpha: 方位角 (°)
    :param P_beta: 傾斜角 (°)
    :param A_col: 集熱器群の面積 (m2)
    :param V_fan_P0: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとしたときの空気搬送ファンの風量 (m3/h)
    :param d0: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の切片 (-)
    :param d1: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の傾き (W/(m2K))
    :param m_fan_test: 集熱器群を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量 (kg/(s・m2))
    :return: 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h)
    """
    from section11_1 import load_outdoor, get_Theta_ex
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # ----- 10 集熱部 -----

    # 空気搬送ファン停止時および稼働時の集熱部の出口における空気温度 (℃)
    Theta_col_nonopg_d_t, Theta_col_opg_d_t = calc_Theta_col(A_col=A_col, P_alpha=P_alpha, P_beta=P_beta,
                                                             V_fan_P0=V_fan_P0, d0=d0, d1=d1, m_fan_test=m_fan_test,
                                                             region=region, sol_region=sol_region,
                                                             Theta_ex_d_t=Theta_ex_d_t)

    # ----- 9. 空気搬送部 -----

    # 1時間当たりの空気搬送ファンの稼働時間 (h/h) (12)
    t_fan_d_t = get_t_fan_d_t(Theta_col_nonopg_d_t, Theta_col_opg_d_t)

    # 1時間当たりの空気搬送ファンの風量 (m3/h) (13)
    V_fan_d_t = get_V_fan_d_t(t_fan_d_t, V_fan_P0)

    # 1時間当たりの空気搬送ファンの消費電力量 (kWh/h) (14)
    E_E_fan_d_t = get_E_E_fan_d_t(fan_sso, fan_type, V_fan_d_t, t_fan_d_t)

    # 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (17)
    r_sa_d_t = get_r_sa_d_t(t_fan_d_t, heating_flag_d)

    # ----- 8. 給湯部 -----

    # 1時間当たりの循環ポンプの稼働時間 (h/h) (6)
    t_cp_d_t = get_t_cp_d_t(hotwater_use, t_fan_d_t, heating_flag_d)

    # ----- 7. 補機の消費電力量 -----

    # 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h) (4)
    E_E_H_aux_ass_d_t = get_E_E_H_aux_ass_d_t(r_sa_d_t, t_cp_d_t, E_E_fan_d_t)

    return E_E_H_aux_ass_d_t


# 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h) (4)
def get_E_E_H_aux_ass_d_t(r_sa_d_t, t_cp_d_t, E_E_fan_d_t):
    """
    :param r_sa_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    :param t_cp_d_t: 1時間当たりの循環ポンプの稼働時間 (h/h)
    :param E_E_fan_d_t: 1時間当たりの空気搬送ファンの消費電力量 (kWh/h)
    :return: 1時間当たりの補機の消費電力量のうち暖房設備への付加分 (kWh/h)
    """

    E_E_H_aux_ass_d_t = np.zeros(24 * 365)

    # (4)
    f = np.logical_and(0 < r_sa_d_t, t_cp_d_t == 0)
    E_E_H_aux_ass_d_t[f] = E_E_fan_d_t[f]

    return E_E_H_aux_ass_d_t


# 1時間当たりの補機の消費電力量のうち給湯設備への付加分 (kWh/h)
def get_E_E_W_aux_ass_d_t(r_sa_d_t, t_cp_d_t, E_E_fan_d_t, E_E_cp_d_t):
    """
    :param r_sa_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    :param t_cp_d_t: 1時間当たりの循環ポンプの稼働時間 (h/h)
    :param E_E_fan_d_t: 1時間当たりの空気搬送ファンの消費電力量 (kWh/h)
    :param E_E_cp_d_t: 1時間当たりの循環ポンプの消費電力量 (kWh/h)
    :return: 1時間当たりの補機の消費電力量のうち給湯設備への付加分 (kWh/h)
    """
    E_E_W_aux_ass_d_t = np.zeros(24 * 365)

    # (5)
    f = np.logical_and(r_sa_d_t == 0, 0 < t_cp_d_t)
    E_E_W_aux_ass_d_t[f] = E_E_fan_d_t[f] + E_E_cp_d_t[f]

    return E_E_W_aux_ass_d_t

### 8．給湯部

#### 8.1　循環ポンプの稼働時間

空気集熱式太陽熱利用設備が給湯部を有さない場合、日付$d$の時刻$t$における1 時間当たりの循環ポンプの稼働時間$t_{cp,d,t}$は、ゼロとする。

空気集熱式太陽熱利用設備が給湯部を有する場合、

$t_{fan,d,t} = 0$、または、日付$d$が暖房日の場合： 

$$\large
t_{cp,d,t} = 0
$$

$0 < t_{fan,d,t}$、または、日付$d$が非暖房日の場合： 

$$\large
t_{cp,d,t} = 1
$$

$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)    
$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)    

In [ ]:
# 1時間当たりの循環ポンプの稼働時間 (h/h) (6)
def get_t_cp_d_t(hotwater_use, t_fan_d_t, heating_flag_d):
    """
    :param hotwater_use: 空気集熱式太陽熱利用設備が給湯部を有する場合はTrue
    :param t_fan_d_t: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    :param heating_flag_d: 暖房日であればTrue
    :return: 1時間当たりの循環ポンプの稼働時間 (h/h)
    """
    t_cp_d_t = np.zeros(24 * 365)

    if hotwater_use == False:
        # (6-1) 集熱した熱を給湯に使用しない場合は循環ポンプは稼働しない
        pass
    else:
        # 暖房日を時間ごとの配列へ展開 365->24*365
        heating_flag_d_t = np.repeat(heating_flag_d, 24)

        # (6-2) 空気搬送ファンが稼働しているが非暖房日の場合は循環ポンプは稼働する
        f = np.logical_and(0 < t_fan_d_t, heating_flag_d_t == False)
        t_cp_d_t[f] = 1.0

    return t_cp_d_t

#### 8.2　循環ポンプの消費電力量

循環ポンプの自立運転用太陽光発電装置を採用する場合、日付$d$の時刻$t$における1 時間当たりの循環ポンプの消費電力量$E_{E,cp,d,t}$は、ゼロとする。

循環ポンプの自立運転用太陽光発電装置を採用しない場合、

$$\large
E_{E,cp,d,t} = P_{cp} \times t_{cp,d,t} \times 10^{-3}
$$

$E_{E,cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの消費電力量(kWh/h)  
$P_{cp}$ : 循環ポンプの消費電力(W)    
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)    

In [ ]:
# 1時間当たりの循環ポンプの消費電力量 (kWh/h) (7)
def get_E_E_cp_d_t(pump_sso, t_cp_d_t):
    """
    :param pump_sso: 循環ポンプの自立運転用太陽光発電装置を採用する場合はTrue
    :param t_fan_d_t: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    :return: 1時間当たりの循環ポンプの消費電力量 (kWh/h)
    """
    if pump_sso == True:
        # 循環ポンプの自立運転用太陽光発電装置を採用する場合は、循環ポンプの消費電力を計上しない
        E_E_cp_d_t = np.zeros(24 * 365)
    else:
        # 循環ポンプの消費電力[W]
        P_cp = get_P_cp()

        # 1時間当たりの循環ポンプの消費電力量 (kWh/h)
        E_E_cp_d_t = P_cp * t_cp_d_t * 10 ** (-3)

    return E_E_cp_d_t

循環ポンプの消費電力$P_{cp}$は、80 W とする。 

In [ ]:
# 循環ポンプの消費電力[W]
def get_P_cp():
    return 80.0

#### 8.3　タンク蓄熱量の上限による補正集熱量

$$\large
\begin{equation*}
   L_{tnk,d} = min \left( Q_{d} , \: HC_{tnk,d} \times \alpha_{tnk,d} \right)
\end{equation*}
$$

$L_{tnk,d}$ : 日付$d$における1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量(MJ/d)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/d)  
$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ)  
$\alpha_{tnk,d}$ : 日付$d$における給湯部のタンク有効利用率(1/d)  

In [ ]:
# 1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量 (MJ/d) (8)
def get_L_tnk_d(Q_d, W_tnk, Theta_wtr_d):
    """
    :param Q_d: 1日当たりの基準集熱量 (MJ/d)
    :param W_tnk: 給湯部のタンク容量 (L)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量 (MJ/d)
    """
    # 給湯部のタンク蓄熱量の上限 (MJ/d) (9)
    HC_tnk_d = get_HC_tnk_d(W_tnk, Theta_wtr_d)

    # 給湯部のタンク有効利用率
    alpha_tnk_d = get_alpha_tnk_d()

    # 1日当たりの給湯部におけるタンク蓄熱量の上限による補正集熱量 (MJ/d) (8)
    L_tnk_d = np.clip(
        Q_d,
        None,
        HC_tnk_d * alpha_tnk_d
    )

    return L_tnk_d

日付$d$における給湯部のタンク有効利用率$\alpha_{tnk,d}$は、日付$d$によらず1.0とする。 

In [ ]:
# 給湯部のタンク有効利用率 [1/d]
def get_alpha_tnk_d():
    # 給湯タンク有効利用率は1.0固定とする
    return np.ones(365)

$$\large
\begin{equation*}
    HC_{tnk,d} = \left( \theta_{tnk} - \theta_{wtr,d} \right) \times W_{tnk} \times 4.186 \times 10^{-3}
\end{equation*}
$$

$HC_{tnk,d}$ : 日付$d$における給湯部のタンク蓄熱量の上限(MJ/d)  
$\theta_{tnk}$ : 給湯部のタンク内温度(℃)   
$\theta_{wtr,d}$ : 日付$d$における日平均給水温度(℃)  
$W_{tnk}$ : 給湯部のタンク容量(L)  

給湯部のタンク内温度$ \theta_{tnk} $は、65℃とする。

In [ ]:
# 給湯部のタンク蓄熱量の上限 (MJ/d) (9)
def get_HC_tnk_d(W_tnk, Theta_wtr_d):
    """
    :param W_tnk: 給湯部のタンク容量 (L)
    :param Theta_wtr_d: 日平均給水温度 (℃)
    :return: 給湯部のタンク蓄熱量の上限 (MJ/d)
    """
    # 給湯部のタンク内温度 (℃)
    Theta_tnk = 65.0

    # 給湯部のタンク蓄熱量の上限 (MJ/d)
    HC_tnk_d = (Theta_tnk - Theta_wtr_d) * W_tnk * 4.186 * 10 ** (-3)

    return HC_tnk_d

#### 8.4　基準集熱量

$$\large
\begin{equation*}
    Q_{d} = \sum_{t=0}^{23} \left( Q_{col,W,d,t} \times f_{s}\right)
\end{equation*}
$$

$f_{s}$ : 給湯部のシステム効率(-)  
$Q_{d}$ : 日付$d$における1日当たりの基準集熱量(MJ/d)  
$Q_{col,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量のうちの給湯利用分(MJ/h)   

給湯部のシステム効率$f_{s}$は、0.85とする。 

In [ ]:
# 1日当たりの基準集熱量 (MJ/d) (10)
def get_Q_d(Q_col_d_t, t_cp_d_t):
    """
    :param Q_col_d_t: 1時間当たりの集熱部における集熱量 (MJ/d)
    :param t_cp_d_t: 1時間当たりの循環ポンプの稼働時間 (h/h)
    :return: 1日当たりの基準集熱量 (MJ/d)
    """
    # 1時間当たりの集熱部における集熱量のうちの給湯利用分 (11)
    Q_col_w_d_t = get_Q_col_w_d_t(Q_col_d_t, t_cp_d_t)

    # 給湯部のシステム効率
    f_s = 0.85

    # 1日当たりの基準集熱量 (MJ/d) (10)
    tmp = Q_col_w_d_t * f_s
    Q_d = np.sum(tmp.reshape(365, 24), axis=1)

    return Q_d

$$\large
\begin{equation*}
    Q_{col,W,d,t} = Q_{col,d,t} \times f_{hx} \times t_{cp,d,t}
\end{equation*}
$$

$f_{hx}$ : 給湯部の熱交換効率(-)  
$Q_{col,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)   
$Q_{col,W,d,t}$ : 日付$d$の時刻$t$における1時間当たりの集熱部における集熱量のうちの給湯利用分(MJ/h)   
$t_{cp,d,t}$ : 日付$d$の時刻$t$における1時間当たりの循環ポンプの稼働時間(h/h)   

給湯部の熱交換効率$f_{hx}$は、0.25とする。 

In [ ]:
# 1時間当たりの集熱部における集熱量のうちの給湯利用分 (MJ/h) (11)
def get_Q_col_w_d_t(Q_col_d_t, t_cp_d_t):
    """
    :param Q_col_d_t: 1時間当たりの集熱部における集熱量 (MJ/d)
    :param t_cp_d_t: 1時間当たりの循環ポンプの稼働時間 (h/h)
    :return: 1時間当たりの集熱部における集熱量のうちの給湯利用分 (MJ/h)
    """
    # 給湯部の熱交換効率
    f_hx = 0.25

    # 1時間当たりの集熱部における集熱量のうちの給湯利用分 (MJ/h) (11)
    Q_col_w_d_t = Q_col_d_t * f_hx * t_cp_d_t

    return Q_col_w_d_t

### 9．空気搬送部 

#### 9.1　空気搬送ファンの稼働時間

$30 \leq \theta_{col,nonopg,d,t}$ かつ $25 < \theta_{col,opg,d,t}$の場合：

$$\large
\begin{equation*}
   t_{fan,d,t} = 1
\end{equation*}
$$

$\theta_{col,nonopg,d,t} < 30$ または $\theta_{col,opg,d,t} \leq 25$の場合：

$$\large
\begin{equation*}
   t_{fan,d,t} = 0
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  

In [ ]:
# 1時間当たりの空気搬送ファンの稼働時間 (h/h) (12)
def get_t_fan_d_t(Theta_col_nonopg_d_t, Theta_col_opg_d_t):
    """
    :param Theta_col_nonopg_d_t: 空気搬送ファン停止時の集熱部の出口における空気温度 (℃)
    :param Theta_col_opg_d_t:  空気搬送ファン稼働時の集熱部の出口における空気温度 (℃)
    :return: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    """
    # 空気搬送ファン稼働時の集熱部の出口における空気温度が25度より大、かつ
    # 空気搬送ファン停止時の集熱部の出口における空気温度が30度以上の場合に稼働する
    t_fan_d_t = np.zeros(24 * 365)
    f = np.logical_and(30 <= Theta_col_nonopg_d_t, 25 < Theta_col_opg_d_t)
    t_fan_d_t[f] = 1

    return t_fan_d_t

#### 9.2　空気搬送ファンの風量

$$\large
\begin{equation*}
   V_{fan,d,t} = V_{fan,P0} \times t_{fan,d,t}
\end{equation*}
$$

$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [ ]:
# 1時間当たりの空気搬送ファンの風量 (m3/h) (13)
def get_V_fan_d_t(t_fan_d_t, V_fan_P0):
    """
    :param t_fan_d_t: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    :param V_fan_P0: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量 (m3/h)
    :return: 1時間当たりの空気搬送ファンの風量 (m3/h)
    """
    V_fan_d_t = V_fan_P0 * t_fan_d_t
    return V_fan_d_t

#### 9.3　空気搬送ファンの消費電力量

空気搬送ファンの自立運転用太陽光発電装置を採用する場合、日付$d$の時刻$t$における1 時間当たりの空気搬送ファンの消費電力量$E_{E,fan,d,t}$は、ゼロとする。 

空気搬送ファンの自立運転用太陽光発電装置を採用しない場合、

$$\large
\begin{equation*}
    E_{E,fan,d,t} =f_{SFP} \times V_{fan,d,t} \times t_{fan,d,t} \times 10^{-3}
\end{equation*}
$$

$E_{E,fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの消費電力量(kWh/h)  
$f_{SFP}$ : 空気搬送ファンの比消費電力(W/(m<sup>3</sup>/h))  
$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  

In [ ]:
# 1時間当たりの空気搬送ファンの消費電力量 (kWh/h) (14)
def get_E_E_fan_d_t(fan_sso, fan_type, V_fan_d_t, t_fan_d_t):
    """
    :param fan_sso: 空気搬送ファンの自立運転用太陽光発電装置を採用する場合はTrue
    :param fan_type: ファンの種別
    :param V_fan_d_t: 1時間当たりの空気搬送ファンの風量 (m3/h)
    :param t_fan_d_t: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    :return: 1時間当たりの空気搬送ファンの消費電力量 (kWh/h)
    """
    if fan_sso == True:
        # 空気搬送ファンの自立運転用太陽光発電装置を採用する場合は、空気搬送ファンの消費電力を計上しない
        E_E_fan_d_t = np.zeros(24 * 365)
    else:
        # 空気搬送ファンの比消費電力 (W/(m3/h))
        f_SFP = get_f_SFP(fan_type)

        # 1時間当たりの空気搬送ファンの消費電力量 (kWh/h) (14)
        E_E_fan_d_t = f_SFP * V_fan_d_t * t_fan_d_t * 10 ** (-3)

    return E_E_fan_d_t

In [ ]:
# 空気搬送ファンの比消費電力 (W/(m3/h))
def get_f_SFP(fan_type):
    """
    :param fan_type: ファンの種別
    :return: 空気搬送ファンの比消費電力 (W/(m3/h))
    """
    if fan_type == 'AC':
        return table_1[0]
    elif fan_type == 'DC':
        return table_1[1]
    else:
        raise ValueError(fan_type)

<div style="text-align: center;font-weight: bold;">表　空気搬送ファンの比消費電力$f_{SFP}$</div>

|ファンの種別|ファンの比消費電力$f_{SFP}$<br/>(W/(m<sup>3</sup>/h))|
|-|-|
|ACファン|0.4|
|DCファン|0.2|


In [ ]:
# 表1 空気搬送ファンの比消費電力 f_SFP
table_1 = (0.4, 0.2)

#### 9.4　床下空間または居室へ供給する空気の風量

$$
\large
V_{sa,d,t,i} = \left\{
\begin{array}{ll}
V_{sa,d,t,A} \times \frac{A_{HCZ,i}}{A_{MR} + A_{OR}} & (i = 1～5 の場合)\\
0 & (i = 6～12 の場合)\\
\end{array}
\right.
$$

$A_{HCZ,i}$ : 暖冷房区画$i$の床面積(m<sup>2</sup>)  
$A_{MR}$ : 主たる居室の床面積(m<sup>2</sup>)  
$A_{OR}$ : その他の居室の床面積(m<sup>2</sup>)  
$V_{sa,d,t,A}$ : 日付$d$の時刻$t$における1時間当たりの床下空間または居室へ供給する空気の風量の合計(m<sup>3</sup>/h)  
$V_{sa,d,t,i}$ : 日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの床下空間または居室へ供給する空気の風量(m<sup>3</sup>/h)  

In [ ]:
# 1時間当たりの床下空間または居室へ供給する空気の風量 (m3/h) (15)
def get_V_sa_d_t_i(i, A_HCZ_i, A_MR, A_OR, V_sa_d_t_A):
    """
    :param i: 暖冷房区画の番号
    :param A_HCZ_i: 暖冷房区画iの床面積 (m2)
    :param A_MR: 主たる居室の床面積 (m2)
    :param A_OR: その他の居室の床面積 (m2)
    :param V_fan_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    :param t_fan_d_t: 1時間当たりの床下空間または居室へ供給する1時間当たりの空気の風量の合計 (m3/h)
    :param heating_flag_d: 暖房日であればTrue
    :param r_sa_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合
    :param V_sa_d_t_A: 床下空間または居室へ供給する1時間当たりの空気の風量の合計
    :return: 1時間当たりの床下空間または居室へ供給する空気の風量 (m3/h)
    """
    if i in [1, 2, 3, 4, 5]:
        return V_sa_d_t_A * A_HCZ_i / (A_MR + A_OR)

    elif i in [6, 7, 8, 9, 10, 11, 12]:
        return np.zeros(24 * 365)

    else:
        raise ValueError(i)

$$
\large
V_{sa,d,t,A} = V_{fan,d,t} \times r_{sa,d,t}
$$

$r_{sa,d,t}$ : 日付$d$の時刻$t$における1 時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合(-)   
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1 時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$V_{sa,d,t,A}$ : 日付$d$の時刻$t$における1時間当たりの床下空間または居室へ供給する空気の風量の合計(m<sup>3</sup>/h)  

In [ ]:
# 床下空間または居室へ供給する1時間当たりの空気の風量の合計 (m3/h) (16)
def get_V_sa_d_t_A(V_fan_d_t, r_sa_d_t):
    """
    :param V_fan_d_t: 1時間当たりの空気搬送ファンの風量 (m3/h)
    :param r_sa_d_t: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    :return: 床下空間または居室へ供給する1時間当たりの空気の風量の合計 (m3/h)
    """
    V_sa_d_t_A = V_fan_d_t * r_sa_d_t

    return V_sa_d_t_A

$t_{fan,d,t} = 0$、または、日付$d$が非暖房日の場合： 

$$\large
r_{sa,d,t} = 0
$$

$0 < t_{fan,d,t}$ かつ日付$d$が暖房日の場合： 

$$\large
r_{sa,d,t} = 1
$$

$r_{sa,d,t}$ : 日付$d$の時刻$t$における1 時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合(-)   
$t_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの稼働時間(h/h)  

In [ ]:
# 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (17)
def get_r_sa_d_t(t_fan_d_t, heating_flag_d):
    """
    :param t_fan_d_t: 1時間当たりの空気搬送ファンの稼働時間 (h/h)
    :param heating_flag_d: 暖房日であればTrue
    :return: 1時間当たりの空気搬送ファンの風量のうち床下空間または居室へ供給する風量の割合 (-)
    """
    # 暖房日を時間ごとの配列へ展開 365->24*365
    heating_flag_d_t = np.repeat(heating_flag_d, 24)

    # 空気搬送ファンの風量が0より大きい、かつ、暖房日である
    r_sa_d_t = np.zeros(24 * 365)
    f = np.logical_and(0.0 < t_fan_d_t, heating_flag_d_t == True)
    r_sa_d_t[f] = 1.0

    return r_sa_d_t

#### 9.5　床下空間または居室へ供給する空気の温度

$$\large
\begin{equation*}
   \theta_{sa,d,t} = \theta_{col,opg,d,t} - \frac{Q_{col,W,d,t}}{\rho_{air} \times c_{p_{air}} \times V_{fan,d,t}}
\end{equation*}
$$

$c_{p_{air}}$： 空気の比熱(kJ/(kg・K))   
$Q_{col,W,d,t}$ : 日付$d$の時刻$t$における1 時間当たりの集熱部における集熱量のうちの給湯利用分(MJ/h)   
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1 時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$：日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{sa,d,t}$：日付$d$の時刻$t$において床下空間または居室へ供給する空気の温度(℃)  
$\rho_{air}$：空気の密度(kg/m<sup>3</sup>)  

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup> とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。 

In [ ]:
# 床下空間または居室へ供給する空気の温度 (℃) (18)
def get_Theta_sa_d_t(V_fan_d_t, Theta_col_opg_d_t, Q_col_W_d_t):
    """
    :param V_fan_d_t: 1時間当たりの空気搬送ファンの風量 (m3/h)
    :param Theta_col_opg_d_t:  空気搬送ファンの稼働時の集熱部の出口における空気温度 (℃)
    :param Q_col_W_d_t: 1時間当たりの集熱部における集熱量のうちの給湯利用分 (MJ/h)
    :return: 床下空間または居室へ供給する空気の温度 (℃)
    """
    # 空気の密度 [kg/m3]
    ro_air = 1.20

    # 空気の比熱 [kJ/(kgK)]
    c_p_air = 1.006

    # 床下空間または居室へ供給する空気の温度 (℃) (18)
    Theta_sa_d_t = np.copy(Theta_col_opg_d_t)
    f = V_fan_d_t > 0
    Theta_sa_d_t[f] = Theta_col_opg_d_t[f] - (Q_col_W_d_t[f] / (ro_air * c_p_air * V_fan_d_t[f]))

    return Theta_sa_d_t

### 10．集熱部 

In [ ]:
# 空気搬送ファン停止時および稼働時の集熱部の出口における空気温度 (℃)
def calc_Theta_col(A_col, P_alpha, P_beta, V_fan_P0, d0, d1, m_fan_test, region, sol_region, Theta_ex_d_t):
    """
    :param A_col: 集熱器群の面積 (m2)
    :param V_fan_P0: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとしたときの空気搬送ファンの風量 (m3/h)
    :param d0: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の切片 (-)
    :param d1: 集熱器群を構成する集熱器の集熱効率特性線図一次近似式の傾き (W/(m2K))
    :param m_fan_test: 集熱器群を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量 (kg/(s・m2))
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 空気搬送ファン停止時および稼働時の集熱部の出口における空気温度 (℃)
    """
    # 集熱器群の数
    n = len(A_col)

    # 作業領域の確保
    V_col_j_d_t = np.zeros((n, 24 * 365))
    Theta_col_nonopg_j_d_t = np.zeros((n, 24 * 365))
    Theta_col_opg_j_d_t = np.zeros((n, 24 * 365))

    for j in range(1, n + 1):
        # 空気搬送ファン稼働時に集熱器群jを流れる空気の体積流量 (m3/h) (24)
        V_col_j_d_t[j - 1, :] = get_V_col_j_d_t(V_fan_P0, A_col[j - 1], A_col)

        # 空気搬送ファン停止時の集熱器群jの出口における空気温度 (℃) (22)
        Theta_col_nonopg_j_d_t[j - 1, :] = get_Theta_col_nonopg_j_d_t(
            P_alpha_j=P_alpha,
            P_beta_j=P_beta[j - 1],
            region=region,
            sol_region=sol_region,
            Theta_ex_d_t=Theta_ex_d_t,
            d0_j=d0[j - 1],
            d1_j=d1[j - 1]
        )

        # 集熱器群jを構成する集熱器の総合熱損失係数 (25)
        U_c_j = get_U_c_j(m_fan_test[j - 1], d1[j - 1])

        # 空気搬送ファン稼働時に集熱器群jを流れる空気の体積流量 (24)
        V_col_j_d_t[j - 1, :] = get_V_col_j_d_t(V_fan_P0, A_col[j - 1], A_col)

        # 空気搬送ファン稼働時の集熱器群jの出口における空気温度 (℃) (23)
        Theta_col_opg_j_d_t[j - 1, :] = get_Theta_col_opg_j_d_t(
            V_col_j_d_t=V_col_j_d_t[j - 1],
            A_col_j=A_col[j - 1],
            U_c_j=U_c_j,
            Theta_col_nonopg_j_d_t=Theta_col_nonopg_j_d_t[j - 1],
            Theta_ex_d_t=Theta_ex_d_t
        )
    # 空気搬送ファン稼働時の集熱部の出口における空気温度 (℃) (21)
    Theta_col_opg_d_t = get_Theta_col_opg_d_t(Theta_col_opg_j_d_t, V_col_j_d_t)

    # 空気搬送ファン停止時の集熱部の出口における空気温度 (℃) (20)
    Theta_col_nonopg_d_t = get_Theta_col_nonopg_d_t(Theta_col_nonopg_j_d_t, V_col_j_d_t)

    return Theta_col_nonopg_d_t, Theta_col_opg_d_t

#### 10.1　集熱量

$$\large
\begin{equation*}
   Q_{col,d,t} = \rho_{air} \times c_{p_{air}} \times V_{fan,d,t} \times \left( \theta_{col,opg,d,t} - \theta_{ex,d,t} \right) \times 10^{-3}
\end{equation*}
$$

$c_{p_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$Q_{col,d,t}$ : 空日付$d$の時刻$t$における1時間当たりの集熱部における集熱量(MJ/h)  
$V_{fan,d,t}$ : 日付$d$の時刻$t$における1時間当たりの空気搬送ファンの風量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup> とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。 

In [ ]:
# 1時間当たりの集熱部における集熱量 (MJ/h) (19)
def get_Q_col_d_t(V_fan_d_t, Theta_col_opg_d_t, Theta_ex_d_t):
    """
    :param V_fan_d_t: 1時間当たりの空気搬送ファンの風量 (m3/h)
    :param Theta_col_opg_d_t: 空気搬送ファン稼働時の集熱部の出口における空気温度 (℃)
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 1時間当たりの集熱部における集熱量 (MJ/h)
    """
    # 空気の密度 [kg/m3]
    ro_air = 1.20

    # 空気の比熱 [kJ/(kgK)]
    c_p_air = 1.006

    # 1時間当たりの集熱部における集熱量 (MJ/h) (19)
    Q_col_d_t = ro_air * c_p_air * V_fan_d_t * (Theta_col_opg_d_t - Theta_ex_d_t) * 10 ** (-3)

    return Q_col_d_t

#### 10.2　空気搬送ファン停止時の集熱部の出口における空気温度

$$\large
\begin{equation*}
   \theta_{col,nonopg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,nonopg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,nonopg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱部の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
# 空気搬送ファン停止時の集熱部の出口における空気温度 (℃) (20)
def get_Theta_col_nonopg_d_t(Theta_col_nonopg_j_d_t, V_col_j_d_t):
    """
    :param Theta_col_nonopg_j_d_t: 空気搬送ファン停止時の集熱器群ごとの出口における空気温度 (℃)
    :param V_col_j_d_t: 空気搬送ファン稼働時に集熱器群ごとに流れる空気の体積流量 (m3/h)
    :return: 空気搬送ファン停止時の集熱部の出口における空気温度 (℃)
    """
    # 集熱器群の数 (-)
    n = Theta_col_nonopg_j_d_t.shape[0]

    # 空気搬送ファン停止時の集熱部の出口における空気温度 (℃)
    Theta_col_nonopg_d_t = \
        np.sum([Theta_col_nonopg_j_d_t[j] * V_col_j_d_t[j] for j in range(n)], axis=0) \
        / np.sum([V_col_j_d_t[j] for j in range(n)], axis=0)

    return Theta_col_nonopg_d_t

#### 10.3　空気搬送ファン稼働時の集熱部の出口における空気温度

$$\large
\begin{equation*}
   \theta_{col,opg,d,t} = \frac{\displaystyle \sum_{j=1}^{n} \left( \theta_{col,opg,j,d,t} \times V_{col,j,d,t} \right)}{\displaystyle \sum_{j=1}^{n} V_{col,j,d,t}}
\end{equation*}
$$

$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,opg,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱部の出口における空気温度(℃)  
$\theta_{col,opg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
# 空気搬送ファン稼働時の集熱部の出口における空気温度 (℃) (21)
def get_Theta_col_opg_d_t(Theta_col_opg_j_d_t, V_col_j_d_t):
    """
    :param Theta_col_opg_j_d_t: 空気搬送ファン稼働時の集熱器群ごとの出口における空気温度 (℃)
    :param V_col_j_d_t: 空気搬送ファン稼働時に集熱器群ごとの空気の体積流量 (m3/h)
    :return: 空気搬送ファン稼働時の集熱部の出口における空気温度 (℃)
    """
    # 集熱器群の数 (-)
    n = Theta_col_opg_j_d_t.shape[0]

    # 空気搬送ファン稼働時の集熱部の出口における空気温度 (℃) (21)
    Theta_col_opg_d_t = \
        np.sum([Theta_col_opg_j_d_t[j] * V_col_j_d_t[j] for j in range(n)], axis=0) \
        / np.sum([V_col_j_d_t[j] for j in range(n)], axis=0)

    return Theta_col_opg_d_t

#### 10.4　空気搬送ファン停止時の集熱器群の出口における空気温度

$$\large
\begin{equation*}
   \theta_{col,nonopg,j,d,t} = \frac{d_{0,j}}{d_{1,j}} ・ I_{s,j,d,t} + \theta_{ex,d,t}
\end{equation*}
$$

$d_{0,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の切片(-)  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$I_{s,j,d,t}$ : 日付$d$の時刻$t$における集熱器群$j$の単位面積当たりの平均日射量(W/m<sup>2</sup>)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  

In [ ]:
# 空気搬送ファン停止時の集熱器群jの出口における空気温度 (℃) (22)
def get_Theta_col_nonopg_j_d_t(P_alpha_j, P_beta_j, region, sol_region, Theta_ex_d_t, d0_j=None, d1_j=None):
    """
    :param P_alpha_j: 方位角 (°)
    :param P_beta_j: 傾斜角 (°)
    :param d0_j: 集熱器群jを構成する集熱器の集熱効率特性線図一次近似式の切片 (-)
    :param d1_j: 集熱器群jを構成する集熱器の集熱効率特性線図一次近似式の傾き (W/(m2K))
    :param region: 省エネルギー地域区分
    :param sol_region: 年間日射地域区分(1-5)
    :return: 空気搬送ファン停止時の集熱器群jの出口における空気温度 (℃)
    """
    # 日射量データの読み込み
    from section11_2 import load_solrad, get_Theta_ex
    solrad = load_solrad(region, sol_region)

    # 単位面積当たりの平均日射量[W/m2]
    I_s_j_d_t = get_I_s_d_t(P_alpha_j, P_beta_j, solrad)

    # 集熱効率特性線図一次近似式の切片および傾きの解決
    if d0_j is None:
        d0_j = get_d0_default()
    if d1_j is None:
        d1_j = get_d1_default()

    # 空気搬送ファン停止時の集熱器群jの出口における空気温度(℃)
    Theta_col_nonopg_j_d_t = d0_j / d1_j * I_s_j_d_t + Theta_ex_d_t

    return Theta_col_nonopg_j_d_t

In [ ]:
# 集熱効率特性線図一次近似式の切片(規定値)[-]
def get_d0_default():
    return 0.1


# 集熱効率特性線図一次近似式の傾き(規定値)[m2K]
def get_d1_default():
    return 2.0

#### 10.5　空気搬送ファン稼働時の集熱器群の出口における空気温度

$$\large
\begin{equation*}
   \theta_{col,opg,j,d,t} = \theta_{col,nonopg,j,d,t} + \left( \theta_{ex,d,t} - \theta_{col,nonopg,j,d,t} \right) \times e^{- \frac{U_{c,j} \times A_{col,j}}{c_{p_{air}} \times \rho_{air} \times V_{col,j,d,t} \div 3600 \times 10^{3}}}
\end{equation*}
$$

$A_{col,j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$c_{p_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$\theta_{col,opg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン稼働時の集熱器群$j$の出口における空気温度(℃)  
$\theta_{col,nonopg,j,d,t}$ : 日付$d$の時刻$t$における空気搬送ファン停止時の集熱器群$j$の出口における空気温度(℃)  
$\theta_{ex,d,t}$ : 日付$d$の時刻$t$における外気温度(℃)  
$\rho_{air}$ : 空気の密度(kg/m<sup>3</sup>)  

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup> とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。 

In [ ]:
# 空気搬送ファン稼働時の集熱器群jの出口における空気温度 (℃) (23)
def get_Theta_col_opg_j_d_t(V_col_j_d_t, A_col_j, U_c_j, Theta_col_nonopg_j_d_t, Theta_ex_d_t):
    """
    :param V_col_j_d_t: 空気搬送ファン稼働時に集熱器群jを流れる空気の体積流量
    :param A_col_j: 集熱器群jの面積 (m2)
    :param U_c_j: 集熱器群jを構成する集熱器の総合熱損失係数
    :param Theta_col_nonopg_j_d_t: 空気搬送ファン停止時の集熱器群jの出口における空気温度 (℃)
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 空気搬送ファン稼働時の集熱器群jの出口における空気温度 (℃)
    """
    # 空気の密度 [kg/m3]
    ro_air = 1.20

    # 空気の比熱 [kJ/(kgK)]
    c_p_air = 1.006

    # 空気搬送ファン稼働時の集熱器群jの出口における空気温度 (℃) (23)
    Theta_col_opg_j_d_t = Theta_col_nonopg_j_d_t + (Theta_ex_d_t - Theta_col_nonopg_j_d_t) * np.exp(
        - (U_c_j * A_col_j) / (c_p_air * ro_air * V_col_j_d_t / 3600 * 10 ** 3))

    return Theta_col_opg_j_d_t

$$\large
\begin{equation*}
   V_{col,j,d,t} = V_{fan,P0} \times \frac{A_{col,j}}{\displaystyle \sum_{j=1}^{n} A_{col,j}}
\end{equation*}
$$

$A_{col,j}$ : 集熱器群$j$の面積(m<sup>2</sup>)  
$n$ : 集熱器群の数(-)  
$V_{col,j,d,t}$ : 日付$d$の時刻$t$における空気集熱ファン稼働時に集熱器群$j$を流れる空気の体積流量(m<sup>3</sup>/h)  
$V_{fan,P0}$ : 空気搬送ファンの送風機特性曲線において機外静圧をゼロとした時の空気搬送ファンの風量(m<sup>3</sup>/h)  

In [ ]:
# 空気搬送ファン稼働時に集熱器群jを流れる空気の体積流量 (m3/h) (24)
def get_V_col_j_d_t(V_fan_P0_j, A_col_j, A_col):
    """
    :param V_fan_P0_j: 空気搬送ファンの送風機特性曲線において機外静圧をゼロとしたときの空気搬送ファンの風量 (m3/h)
    :param A_col_j: 集熱器群jの面積 (m2)
    :param A_col: 集熱器群の面積 (m2)
    :return: 空気搬送ファン稼働時に集熱器群jを流れる空気の体積流量 (m3/h)
    """
    V_col_j_d_t = V_fan_P0_j * A_col_j / sum(A_col)
    return V_col_j_d_t

$$\large
\begin{equation*}
   U_{c,j} = -c_{p_{air}} \times \dot{m}_{fan,test,j} \times 10^{3} \times ln \left( 1 - \frac{1}{c_{p_{air}} \times \dot{m}_{fan,test,j} \times 10^{3}} \times d_{1,j}  \right)
\end{equation*}
$$

$c_{p_{air}}$ : 空気の比熱(kJ/(kg$\cdot$K))  
$d_{1,j}$ : 集熱器群$j$を構成する集熱器の集熱効率特性線図一次近似式の傾き(W/(m<sup>2</sup>$\cdot$K))  
$\dot{m}_{fan,test,j}$ : 集熱器群$j$を構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量(kg/(s$\cdot$m<sup>2</sup>))  
$U_{c,j}$ : 集熱器群$j$を構成する集熱器の集熱器総合熱損失係数(W/(m<sup>2</sup>$\cdot$K))  

空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。 

In [ ]:
# 集熱器群jを構成する集熱器の総合熱損失係数 (25)
def get_U_c_j(m_fan_test_j=None, d1_j=None):
    """
    :param m_fan_test_j: 集熱器群jを構成する集熱器の集熱性能試験時における単位面積当たりの空気の質量流量 (kg/(s・m2))
    :param d1_j: 集熱器群jを構成する集熱器の集熱効率特性線図一次近似式の傾き (W/m2K)
    :return: 集熱器群jを構成する集熱器の総合熱損失係数
    """
    # 空気の比熱 [kJ/(kgK)]
    c_p_air = 1.006

    if m_fan_test_j is None:
        m_fan_test_j = 0.0107
    if d1_j is None:
        d1_j = 2.0

    # 集熱器群jを構成する集熱器の総合熱損失係数 (25)
    U_c_j = - c_p_air * m_fan_test_j * 10 ** 3 * np.log(1 - 1 / (c_p_air * m_fan_test_j * 10 ** 3) * d1_j)

    return U_c_j

## 付録 A ~ F

省略